In [24]:
#import networkx as nx
import csv
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib;
import pandas as pd 
import utm
import matplotlib.pyplot as plt 
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
from ipyleaflet import Map, FullScreenControl, Polyline
from datetime import datetime, timedelta, timezone
import subprocess
import numpy as np
ox.config(use_cache=True, log_console=True)
%matplotlib inline
from shapely.geometry import Point
import pyproj, math

class GpsObs:

    def __init__(self, data):
         # Class attributes
        self.time = data[1]
        self.name = data[2]
        self.satNum = data[3]
        self.carrier1 = data[5] # carrier-phase cycle 1
        self.carrier2 = data[6] # carrier-phase cycle 2
        self.prng1 = data[7] # pseudorange freq 1
        self.prng2 = data[8] # pesudorange freq 2
        self.doppler1 = data[9] # doppler frequency [Hz]
        self.doppler2 = data[10] # doppler frequency [Hz]
        self.satPos = [0, 0, 0]
       # self.occupGrid = OccupancyGrid()            # Generated Occupancy Grid  

gps_trace_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/results.pos.trace"


# Table of dictionaries containing multiple obs_set
obs_tab = []
prng_set = {}
with open(gps_trace_file_path, 'r') as f:
    reader = csv.reader(f, delimiter='\n')
    data_trace = list(reader)
    idx_obs_s = -1
    idx_obs_e = -1
    idx_nav_s = -1
    idx_nav_e = -1
    # Set of multiple GPS observations (per given time)
    obs_set = {}

    for i, line in enumerate(data_trace[:1000]):
        # Convert array to string
        line = line[0]
        # Split everything in the same way
        line = line.split('/')[-1]
        if '#OBSERVATIONS_START' in line:
            idx_obs_s = i
        if '#OBSERVATIONS_END' in line:
            idx_obs_e = i
            # Add obs_set to the table and reset its value
            obs_tab.append(obs_set)
            obs_set={}
        if '#SATPOSITION_START' in line:
            idx_nav_s = i
        if '#SATPOSITION_END' in line:
            idx_nav_e = i
            
        if 'PRNG' in line:
            line2 = line.split()
            if line2[2] not in prng_set:
                prng_set[line2[2]] = [line2[3], line2[4]]
        
        if i > idx_obs_s and ((i < idx_obs_e) or (idx_obs_e < idx_obs_s)):
            line = line.split()
            if line[4] == 'rcv2':  # rcv2 - stationary base receiver
                continue
            new_obs = GpsObs(line)
            obs_set[new_obs.satNum] = new_obs
            
        if i > idx_nav_s and ((i < idx_nav_e) or (idx_nav_e < idx_nav_s)):
            line = line.split()
            satNum = line[2]
            obs = obs_tab[-1]
            if(satNum in obs):
                obs[satNum].satPos = [float(line[3]), float(line[4]), float(line[5])]

for key, value in  prng_set.items():
    print (key, value)
    
out_path = '/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/gps.g2o'
with open(out_path, 'w') as f:

    # Initial vertex # number, pose, quaternion
    f.write('VERTEX_SE3:QUAT 0 0 0 0 0 0 0 1\n')
    v_idx = 0
    e_idx = -0
    for obs_set in obs_tab:
        for key, value in  obs_set.items():
            v_idx += 1
            if(key in prng_set):
                if (float(prng_set[key][0]) > 0):
                    f.write('VERTEX_SE3:QUAT {} {} {} {} 0 0 0 1\n'.format(v_idx, value.satPos[0], value.satPos[1], value.satPos[2]))
                    f.write('FIX {}\n'.format(v_idx))
                    f.write('GPSEDGE 0 {} {} {}\n'.format(v_idx, prng_set[key][0], 1.0/math.sqrt(float(prng_set[key][1]))))
                    #f.write('GPSEDGE 0 {} {} {}\n'.format(v_idx, value.prng1, 1.0/math.sqrt(float(prng_set[key][1]))))

        for key, value in  prng_set.items():
            e_idx += 1
            # Edge from, to, distance, noise
            noise = 1000
            # f.write('GPSEDGE 0 {} {} {}\n'.format(e_idx, value, 1.0/noise))

        for key, value in  obs_set.items():
            pass
        
        # Only first for test
        break;
        
        
g2o_result_path = '/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/g2o_result.txt'
subprocess.call("./MyGPS", shell=True, cwd="/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/")
with open(g2o_result_path, 'r') as f:
    res = f.readline().split()
    print(res)
    

ecef_gpd = gpd.points_from_xy([res[0]], [res[1]], z=[res[2]], crs="EPSG:4978")
lon_lat_gpd = ecef_gpd.to_crs(crs="EPSG:4326")
print(lon_lat_gpd[0].y, lon_lat_gpd[0].x, lon_lat_gpd[0].z )

gps_result_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/results.pos"
point = [lon_lat_gpd[0].y, lon_lat_gpd[0].x]
m = Map(center=point, basemap=basemaps.OpenStreetMap.Mapnik, zoom=15)
marker = Marker(location=point, draggable=False)
rtklib_est_ecef = [-3963425.455 ,3350879.447 ,3694864.328]
rtklib_est_ecef_gpd = gpd.points_from_xy([rtklib_est_ecef[0]], [rtklib_est_ecef[1]], z=[rtklib_est_ecef[2]], crs="EPSG:4978")
rtklib_est_lonlat = rtklib_est_ecef_gpd.to_crs(crs="EPSG:4326")
marker2 = Marker(location=(rtklib_est_lonlat[0].y, rtklib_est_lonlat[0].x), draggable=True)
m.add_layer(marker)
m.add_layer(marker2)
m

5 ['23437832.318', '25.573']
15 ['20423326.127', '11.116']
20 ['22427720.129', '17.802']
21 ['22748307.140', '17.611']
24 ['20203399.599', '10.900']
38 ['21493681.082', '38.386']
39 ['19788581.439', '30.879']
40 ['21947476.693', '39.459']
41 ['19455944.155', '30.312']
48 ['22514441.562', '44.467']
60 ['24484585.850', '16.347']
63 ['23844638.599', '14.563']
68 ['24598956.715', '16.496']
90 ['24626837.881', '17.206']
96 ['38298211.980', '16.396']
97 ['37065618.469', '17.887']
108 ['37975615.444', '17.111']
109 ['37577304.970', '15.574']
111 ['36841348.169', '11.819']
113 ['38926431.574', '24.942']
114 ['38345733.686', '17.111']
119 ['22038001.052', '11.498']
121 ['36925430.057', '12.620']
['-3963439.33031008', '3352151.1319114', '3693151.39956308']
35.61240816184291 139.7765191658873 -279.81084527168423


Map(center=[35.61240816184291, 139.7765191658873], controls=(ZoomControl(options=['position', 'zoom_in_text', …